### LlamaIndexの前準備

In [1]:
# パッケージのインストール
!pip install llama-index==0.10.39
!pip install llama-index-llms-gemini
!pip install llama-index-embeddings-huggingface

INFO: pip is looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-parse to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 374.3 kB/s eta 0:00:00a 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 602.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 425.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [4]:
# 環境変数の準備 (左端の鍵アイコンでGOOGLE_API_KEYを設定)
import google.generativeai as genai
from dotenv import load_dotenv
import os
from tqdm.notebook import tqdm

load_dotenv()
GOOGLE_API_KEY=os.environ.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
import logging
import sys

# ログレベルの設定
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)

In [8]:
from llama_index.core import Settings
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# LLMの準備
Settings.llm = Gemini(
    model_name="models/gemini-1.5-flash",
    safety_settings={
        "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
        "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
        "HARM_CATEGORY_SEXUALLY_EXPLICIT" : "BLOCK_NONE",
        "HARM_CATEGORY_DANGEROUS_CONTENT" : "BLOCK_NONE"
    }
)

# 埋め込みモデルの準備
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config_sentence_transformers.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/README.md HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/modules.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/sentence_bert_config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /BAAI/bge-m3/resolve/main/pytorch_model.bi

/usr/local/lib/python3.11/site-packages/huggingface_hub/file_download.py:653: UserWarning: Not enough free disk space to download the file. The expected file size is: 2271.06 MB. The target location /tmp/llama_index/models--BAAI--bge-m3/blobs only has 2236.06 MB free disk space.
  warnings.warn(


DEBUG:urllib3.connectionpool:https://cdn-lfs-us-1.hf.co:443 "GET /repos/23/2c/232ca60237b0bb19bb6c28c5a6c8af79f2e423333a9626aad445543b80fbf31e/993b2248881724788dcab8c644a91dfd63584b6e5604ff2037cb5541e1e38e7e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1731414133&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMTQxNDEzM319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzIzLzJjLzIzMmNhNjAyMzdiMGJiMTliYjZjMjhjNWE2YzhhZjc5ZjJlNDIzMzMzYTk2MjZhYWQ0NDU1NDNiODBmYmYzMWUvOTkzYjIyNDg4ODE3MjQ3ODhkY2FiOGM2NDRhOTFkZmQ2MzU4NGI2ZTU2MDRmZjIwMzdjYjU1NDFlMWUzOGU3ZT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=lrMKBBXngb~VWzu7r9gELbmBAxbGzPWQ23Iz-KgphloH5NW~9K8dvuy9AVlTqPL53BVuT~f~LBGl~EvUFqRu1fPNOlT7uVKvOpNbc3ikjUcDEWrAt8eZZ6fK1DrmAhSVlvYDU~ujYP09nltxCOeTfUMG7JecqsoYZR-MqccgrcA6vV9ChfwU2R8dZr2-DWCxw1W4symF5xNb~1MuXzabjGnO7Vmg-DkCXfuKDdgs93SP7uekjwbj3EAd9p

### LlamaIndexへの質問応答

In [9]:
from llama_index.core import SimpleDirectoryReader

# ドキュメントの読み込み (dataフォルダにドキュメントを配置しておきます)
documents = SimpleDirectoryReader("data").load_data()

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 7
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin1.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin2.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin3.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin4.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin5.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin6.txt
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/data/akazukin7.txt


In [10]:
documents

[Document(id_='06af5b43-3609-4412-a51b-367099bdf9a5', embedding=None, metadata={'file_path': '/work/gemini_npaka/llamaindex/data/akazukin1.txt', 'file_name': 'akazukin1.txt', 'file_type': 'text/plain', 'file_size': 632, 'creation_date': '2024-11-09', 'last_modified_date': '2024-11-09'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='第1章：ネオン街の影\n2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。\nアズキのターゲットは、企業連合「帝国」の秘密情報。帝国は街を支配し、人々を搾取していた。アズキたちは、そんな帝国に立ち向かうレジスタンスの尖兵だった。\nしかし、アズキは罠に嵌められた。背後から襲い掛かる帝国の警備隊。アズキは必死に逃げ回るが、追いつかれ、捕らわれてしまう。\n', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='3a6c80e7-c186-428b

In [11]:
from llama_index.core import VectorStoreIndex

# インデックスの作成
index = VectorStoreIndex.from_documents(documents)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めな...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第3章：情報収集
ネオン街の裏社会に潜伏するアズキとオオカミ。彼らは帝国の秘密情報を収集するた...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキは...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第5章：決戦
ついに、アズキとオオカミは帝国の本拠地へ潜入する。そこで、彼らは帝国の恐るべき計...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第6章：希望の光
激しい戦いの末、アズキとオオカミはシステムを破壊することに成功する。帝国は崩...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 第7章：新たな旅
街に平和が訪れた数年後、アズキは新たな旅に出る。
彼女は世界中を旅し、他の抑...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


In [12]:
index

In [13]:
# クエリエンジンの作成
query_engine = index.as_query_engine()

In [14]:
# 質問応答
print(query_engine.query("アズキの年齢は？"))

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.10it/s]

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 4e074c74-b43b-4a89-95f4-bd3ba65f766f] [Similarity score:             0.544501] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...
> [Node ecd98ce1-aa57-4d2d-90de-69727517db9a] [Similarity score:             0.510709] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...


アズキは16歳です。 



In [15]:
# 質問応答
print(query_engine.query("アズキが牢獄で出会ったハッカーの名前は？"))

Batches: 100%|██████████| 1/1 [00:00<00:00,  6.09it/s]

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node ecd98ce1-aa57-4d2d-90de-69727517db9a] [Similarity score:             0.705717] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...
> [Node 4e074c74-b43b-4a89-95f4-bd3ba65f766f] [Similarity score:             0.597005] 第1章：ネオン街の影
2078年、東京。ネオン街の闇に紛れ、少女アズキはフードを深く被り、路地裏を疾走していた。彼女は16歳、ハッカー集団「紅ずきん」の一員だ。
アズキのターゲットは、企業連合「...


オオカミ 



In [16]:
# 質問応答
print(query_engine.query("オオカミがアズキを裏切った理由は？"))

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.08it/s]

DEBUG:llama_index.core.indices.utils:> Top 2 nodes:
> [Node 0fe1724c-9b83-4aea-877e-c2f600b353bd] [Similarity score:             0.642775] 第4章：裏切り
ある日、オオカミの裏切りが発覚する。彼は帝国に情報を売っていたのだ。
アズキはショックを受け、オオカミを問い詰める。オオカミは、妹を人質に取られており、帝国に協力せざるを得なかっ...
> [Node ecd98ce1-aa57-4d2d-90de-69727517db9a] [Similarity score:             0.555725] 第2章：牢獄の出会い
暗い牢獄に閉じ込められたアズキ。絶望に打ちひしがれながらも、彼女は諦めなかった。脱出する方法を探るアズキの前に、謎めいた青年が現れる。
青年はハッカー「オオカミ」と呼ばれ、...


オオカミは妹が人質に取られており、帝国に協力せざるを得なかった。 



### インデックスの保存と読み込み

In [17]:
# インデックスの保存
index.storage_context.persist()

DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/docstore.json
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/index_store.json
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/graph_store.json
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/default__vector_store.json
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/image__vector_store.json


In [18]:
from llama_index.core import StorageContext, load_index_from_storage

# インデックスの読み込み
storage_context = StorageContext.from_defaults(persist_dir="./storage")
index = load_index_from_storage(storage_context)

DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/docstore.json.
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/docstore.json
DEBUG:llama_index.core.storage.kvstore.simple_kvstore:Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage/index_store.json.
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/index_store.json
DEBUG:llama_index.core.graph_stores.simple:Loading llama_index.core.graph_stores.simple from ./storage/graph_store.json.
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/graph_store.json
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/property_graph_store.json
DEBUG:llama_index.core.vector_stores.simple:Loading llama_index.core.vector_stores.simple from ./storage/image__vector_store.json.
DEBUG:fsspec.local:open file: /work/gemini_npaka/llamaindex/storage/image__vector_store.json
DEBUG:llama_index.core.vec